# Final Project

by Sayan Chatterjee, Marcell Gilliam, Andrew Henderson

# Description

Purpose: This is for the CS455 final project. A genetic algorithm to control the gains of a satellite.

In [2]:
from statistics import mean
from itertools import chain

class geneticAlgorithm:
    
    def __init__(self, matrixPopulation, generations, cross, mutate, Ra):
        self.population = matrixPopulation
        self.gen = generations
        self.crossProb = cross
        self.mutateProb = mutate
        
        matrixFitness = [0] * len(self.population)
        popSize = len(matrixPopulation)
        for i in range(self.gen):
            '''
            #1 get fitness for each matrix by calling findEig and calcFitness
                append fitness into a list that corresponds with matrix
            '''
            for j in range(len(self.population)):
                sat.implementController(self.population[j])
                sat.findEig() #self.population[j]
                matrixFitness[j] = sat.calculateFitness() #send matrix to get fitness
                print(matrixFitness)
            
            '''
            Print fitness Results for each generation?
            '''
            
            '''
            #2 Use getBest to pop out low performing fitnesses
            '''
            
            fitPopulation = self.getBest(matrixFitness, self.population)
            
            '''
            #3 rebuildPopulation with remaining matrices
               determine probability of crossovers before parent selection
               Determine probability of mutation before chromosome selection
            '''
            self.population = self.rebuildPopulation(fitPopulation, popSize, Ra)
            print("Rebuilt K Population", self.population)
    
    def getBest(self, fitness, population):
        #take average of fitness, then create threshold
        fitPopulation = []
        threshold = mean(fitness)
        
        matrixSize = len(fitness)
        
        for i in range(matrixSize):
            if fitness[i] > threshold:
                #fitness.pop(i)
                fitPopulation.append(population[i])
        #print("Old fit Population", population)
        #print("New fit Population", fitPopulation)
        return(fitPopulation)
        
    def rebuildPopulation(self, currPopulation, popSize, Ra):
        #print(self.crossProb, self.mutateProb)
        refillSize = popSize - len(currPopulation)
        for i in range(refillSize):
            #fills in 1 matrix per iteration and appends
            newMatrix = []
            j=0

            while j < Ra:#change to adjust to matrix size Ra^2
                #one chromosome per iteration
                newChromosome = []
                if np.random.uniform(0,1) < self.crossProb:
                
                    ##convert into for loop for varying size matrices
                    parent1, parent2 = self.parentSelect()
                    parent1 = sat.int2bin(parent1)
                    parent2 = sat.int2bin(parent2)
                
                    bitPivot = self.chromosomeSelect()
                    #crossover
                
                    newChromosome.append(parent1[:bitPivot])
                    newChromosome.append(parent2[bitPivot:])
                    #print("2D?",newChromosome)
                    # New Gene is created: newChromosome
                    newChromosome = list(chain.from_iterable(newChromosome)) #flattens the list to 1D
                    #print("Current NewChromo @ len(", len(newChromosome), ")", newChromosome)
                    if np.random.uniform(0,1) < self.mutateProb:
                        bit = random.randint(0,31)
                        if newChromosome[bit] == 0:
                            newChromosome[bit] = 1
                        else:
                            newChromosome[bit] = 0
                    #
                    
                    #
                    #print("newChromosome:", newChromosome, len(newChromosome))
                    newGene = sat.bin2number(newChromosome)
                    #print("newChrom -> newGene:", newGene)
                else:
                    #Create a new chromosome (number)
                    newGene = np.random.uniform(-2**15,2**15)
                    #print("Just newGene:", newGene)
                newMatrix.append(newGene)
                #print("filling matrix:", newMatrix)
                j+=1
                #end while
            #print("New Matrix:", newMatrix)
            #newMatrix = np.reshape(newMatrix, (2,2)) #change to (Ra, Ra)
            currPopulation.append(newMatrix)
        #end for
        return(currPopulation)

    def parentSelect(self):
        
        #Choose a random parent
        matrix1 = random.choice(self.population)
        matrix2 = random.choice(self.population)
        chromosome1 = random.choice(matrix1)
        chromosome2 = random.choice(matrix2)
        
        return chromosome1, chromosome2
    
    def chromosomeSelect(self):
        return random.randint(0,31)

    def plotScores(self):
        plt.plot(self.bests) #Takes best fitness of a matrix from each generation
        plt.plot(self.averages) #Takes fitness of each matrix and divide by popSize
        plt.ylim(bottom=0)
        plt.show()
        

In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.linalg as la
import control
from control.matlab import *

class satellite:
    def __init__(self, A, B, C, D):
        # Following the Form x' = Ax +Bu; y = Cx+Du
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        self.eigvals = []
        self.eigvecs = []
        self.fitness = 0 # Represents the fitness Value
        self.sys1 = [] # Represents the Transfer Function
        self.wn = []#Represents the transfer function
        self.damping = [] #Represents the damping coefficients
        self.poles = [] #Represents the location of the poles
        
    def printMatrix(self):
        print("Printing the A and B Matrix\n")
        print(self.A)
        print(self.B)
    
    def findEig(self):
        self.eigvals, self.eigvecs = la.eig(A)
        
    def getEigVal(self):
        return self.eigvals
    def implementController(self, K):
        self.A = (self.A-self.B*K)
##############################################################################################
    def calculateFitness(self):
        for y in np.nditer(self.eigvals, op_flags=['readwrite']):
           if y > 0:
               self.fitness = self.fitness+0 #Only Symbolic to show unstability
           elif y == 0:
               self.fitness = self.fitness+0.5 #Marginally Stable
                #Edit(meg): constant was originally 0
           elif y < 0:
               self.fitness = self.fitness+2 #Stable
               
        #Creating Transfer Function for Further Analysis
        self.sys1 = control.ss2tf(self.A,self.B,self.C,self.D)
        self.wn, self.damping, self.poles = control.damp(self.sys1, doprint =True)
        
        for i in self.wn:
            if i <= 1 and i >= -1: #There is a little play regarding angular velocity
                self.fitness +=1 #If the angular velocity is zero, the satellite is stationary and stable
        else:
            self.fitness +=0 #If the angular velocity is numerical, then it is marginally stable
        return self.fitness
##############################################################################################    
    def int2bin(self, decVal):
        binary = [0] * 32
        binPos = 15
        
        if decVal < 0:
            binary[0] = 1 #indicates negative
            decVal = decVal * (-1)

        pointVal = str(decVal-int(decVal)) #subtracts numbers after the decimal to only have integer
        pointVal = float(pointVal)

        decVal = decVal - pointVal

        while decVal > 0:
            binary[binPos] = int(decVal)%2
            decVal = int(decVal)//2
            binPos = binPos - 1

        #print(binary)
        #append to some list
        return(self.dec2bin(pointVal, binary))

    def dec2bin(self, pointVal, binary):
        binPos = 16 #16 -> 31

        while binPos < 32:
            pointVal = pointVal * 2

            if pointVal >= 1:
                binary[binPos] = 1
                pointVal = pointVal-1

            binPos = binPos + 1
        #print(binary)
        return binary
        #bin2number(binary)
        #append to binary number

    def bin2number(self, binary):
        #binary = [1] *32
        wholeNum = 0
        binPos = 16
        # whole number
        # 1 -15; count down from 15
        exp = 0
        while binPos < 31:
            if binary[binPos] == 1:
                wholeNum += 2**exp
            exp += 1
            binPos+=1

        fracNum = 0
        binPos = 15
        exp = -1

        while binPos >= 0:
            if binary[binPos] == 1:
                fracNum += 2**exp
            exp -= 1
            binPos -=1

        number = round((wholeNum + fracNum), 3)
        if binary[0] == 1:
            number *= -1
        #print(number)
        return(number)
    
    def buildPopulation(self, popSize, initialMatrix, Ra):
        
        #initialMatrix = np.reshape(initialMatrix, 4, order='F')
        #print(initialMatrix[0])
        population = []
        tempMatrix = []
        
        population.append(initialMatrix)

        for matrix in range(popSize-1): #1 matrix at a time
            
            matrixA = []
            chromosome = []
            
            for index in range((Ra)):
                
                newNum = round(np.random.uniform((-10), (10)), 2)
                chromosome.append(newNum)
                
            #matrixA.append(chromosome)
            
            #matrixA = np.reshape(matrixA, (Ra,Ra)) #change to Ra
            
            population.append(chromosome)
            ###
        return(population)
    
        #enter generations
        '''
        get fit!
        '''
        

ModuleNotFoundError: No module named 'control'

# Genetic Section

Description: Here will be the implementation of the genetic algorithm. Each binary string will represent the control matrix

# The start of the theoretical Main Method

"This is Data Input"

In [ ]:
##################################################################
Ra = 3 #int(input("Enter the number of rows for A Matrix:")) #Rows for Matrix A
Ca = 3 #int(input("Enter the number of columns for A Matrix:")) #Rows for Matrix B
print("Enter the entries in a single line (separated by space): ") 
  
# User input of entries in a single line separated by space 
entries = [-51.5, 10.63, 79.015, -25, 5, 12, 7.2, 1.1, 4.2] #list(map(float, input().split())) 
  
# For printing the matrix A
A = np.array(entries).reshape(Ra, Ca)

#Rb = int(input("Enter the number of rows for B Matrix:")) 
#Cb = int(input("Enter the number of columns for B Matrix:")) 
Rb = Ra
Cb = 1
print("Enter the entries in a single line (separated by space) for B Matrix(Size {}): ".format(Rb*Cb)) 
  
# User input of entries in a  
# single line separated by space 
entries = [15.9, -9.29, 12] # list(map(float, input().split())) 
  
# For printing the matrix 
B = np.array(entries).reshape(Rb, Cb)

Rc = 1
Cc = Ra
print("Enter the entries in a single line (separated by space) for C Matrix(Size {}): ".format(Rb*Cb))

D = [0] 
  
# User input of entries in a  
# single line separated by space 
entries = [5.63, -4.25, 14] #list(map(float, input().split())) 
  
# For printing the matrix 
C = np.array(entries).reshape(Rc, Cc)
#################################################################################################

# Creating the Object
sat = satellite(A, B, C, D)
'''
call build population here pass on A
'''
# sat.printMatrix() #Used for testing only

#####sat.findEig() # find eigenvalues of each matrix to get fitness of matrix

# print(sat.getEigVal()) #Used only for testing
#####print("Fitness", sat.calculateFitness()) #Used for Testing Only
# print(control.ss2tf(A, B, C, D)) #Used for testing only

# Creating the Controller Design
K = np.zeros(Ra) # K represents the gain. The signal row has to match the col of A Matrix
'''
start here and run GA on K
'''
popSize = 5
matrixPopulation = sat.buildPopulation(popSize, K, Ra)

#print("This is K:", matrixPopulation)

generations = 10
cross = 0.5
mutate = 0.035

geneticAlgorithm(matrixPopulation, generations, cross, mutate, Ra)
print("Program End")
